baseline net with no hidden layers

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the input data to [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten the images (28x28 -> 784)
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create a model with no hidden layers
model = Sequential([
    Dense(10, activation='softmax', input_shape=(784,))  # Output layer with 10 neurons (one for each digit)
])

# Compile the model
model.compile(optimizer='sgd',  # Stochastic Gradient Descent
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


2024-12-18 18:18:37.115591: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-18 18:18:37.116522: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-18 18:18:37.118828: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-18 18:18:37.124769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734545917.136617   23663 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734545917.13

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/home/kajm20/miniconda3/envs/rmnist/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-18 18:18:40.296676: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 776us/step - accuracy: 0.6703 - loss: 1.2417 - val_accuracy: 0.8760 - val_loss: 0.5117
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 701us/step - accuracy: 0.8695 - loss: 0.5181 - val_accuracy: 0.8893 - val_loss: 0.4200
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 809us/step - accuracy: 0.8833 - loss: 0.4380 - val_accuracy: 0.8982 - val_loss: 0.3824
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 734us/step - accuracy: 0.8893 - loss: 0.4076 - val_accuracy: 0.9023 - val_loss: 0.3617
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 786us/step - accuracy: 0.8960 - loss: 0.3852 - val_accuracy: 0.9061 - val_loss: 0.3487
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 725us/step - accuracy: 0.8972 - loss: 0.3679 - val_accuracy: 0.9075 - val_loss: 0.3386
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 701us/step - accuracy: 0.9023 - loss: 0.3498 - val_accuracy: 0.9094 - val_loss: 0.3305
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 714us/step - accuracy: 0.9003 -

rl net

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import numpy as np
import random

# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0  # Normalize input to [0, 1]
x_test = x_test / 255.0
y_train = to_categorical(y_train, 10)  # One-hot encode labels
y_test = to_categorical(y_test, 10)

# RL hyperparameters
MAX_HIDDEN_LAYERS = 5
MAX_NEURONS = 512
EPISODES = 20

# Reward function parameters
ACCURACY_WEIGHT = 100
COMPLEXITY_PENALTY = 0.01

# Function to create and train a model with a given architecture
def create_and_train_model(hidden_layers):
    # Create a model
    model = Sequential([Flatten(input_shape=(28, 28))])  # Input layer
    for neurons in hidden_layers:
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(10, activation='softmax'))  # Output layer

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=3, batch_size=32, verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

# RL agent to explore architectures
for episode in range(EPISODES):
    # Randomly decide the number of hidden layers and their neurons
    num_hidden_layers = random.randint(0, MAX_HIDDEN_LAYERS)
    hidden_layers = [random.randint(10, MAX_NEURONS) for _ in range(num_hidden_layers)]
    
    # Train and evaluate the model
    accuracy = create_and_train_model(hidden_layers)

    # Calculate the reward
    complexity = sum(hidden_layers)
    reward = (accuracy * ACCURACY_WEIGHT) - (complexity * COMPLEXITY_PENALTY)

    # Log results
    print(f"Episode {episode + 1}:")
    print(f"  Hidden Layers: {len(hidden_layers)}")
    print(f"  Neurons: {hidden_layers}")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Complexity: {complexity}")
    print(f"  Reward: {reward:.2f}")
    print()


/home/kajm20/miniconda3/envs/rmnist/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Episode 1:
  Hidden Layers: 4
  Neurons: [29, 407, 504, 328]
  Accuracy: 0.9660
  Complexity: 1268
  Reward: 83.92

Episode 2:
  Hidden Layers: 4
  Neurons: [183, 436, 473, 79]
  Accuracy: 0.9706
  Complexity: 1171
  Reward: 85.35

Episode 3:
  Hidden Layers: 5
  Neurons: [275, 427, 113, 449, 42]
  Accuracy: 0.9729
  Complexity: 1306
  Reward: 84.23

Episode 4:
  Hidden Layers: 0
  Neurons: []
  Accuracy: 0.9232
  Complexity: 0
  Reward: 92.32

Episode 5:
  Hidden Layers: 5
  Neurons: [438, 311, 488, 17, 202]
  Accuracy: 0.9755
  Complexity: 1456
  Reward: 82.99

Episode 6:
  Hidden Layers: 1
  Neurons: [118]
  Accuracy: 0.9730
  Complexity: 118
  Reward: 96.12

Episode 7:
  Hidden Layers: 4
  Neurons: [308, 368, 31, 49]
  Accuracy: 0.9713
  Complexity: 756
  Reward: 89.57

Episode 8:
  Hidden Layers: 0
  Neurons: []
  Accuracy: 0.9246
  Complexity: 0
  Reward: 92.46

Episode 9:
  Hidden Layers: 2
  Neurons: [308, 264]
  Accuracy: 0.9757
  Complexity: 572
  Reward: 91.85

Episode 10:
 